In [1]:
import numpy as np
import pandas as pd
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
from classes._env3W import env3W  # Ajuste para o caminho correto da sua classe de ambiente

# 1. Instalar Dependências

In [ ]:
pip install tf-agents tensorflow


# Definir a Rede Q

In [18]:
from tf_agents.networks import q_network
from tf_agents.utils import common

df = pd.read_csv('df_env.csv')
environment = env3W(df)
fc_layer_params = (100, 50)  # Exemplo de parâmetros das camadas totalmente conectadas

q_net = q_network.QNetwork(
    environment.observation_spec(),
    environment.action_spec(),
    fc_layer_params=(100, 50))



# Configurar o Agente DQN

In [19]:
import tensorflow as tf
from tf_agents.agents.dqn import dqn_agent

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=1e-3)
# Exemplo de criação da instância do ambiente
# Supondo que 'dataframe' seja o seu DataFrame pandas


train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    environment.time_step_spec(),
    environment.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()


# Treinar o Agente

In [20]:
from tf_agents.trajectories import trajectory

import numpy as np

def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    
    # Supõe que 'observation' é a observação atual do timestep e precisa ser ajustada
    # para incluir a dimensão do batch.
    observation = np.expand_dims(time_step.observation, axis=0)
    
    # Cria um novo timestep com a observação ajustada.
    adjusted_time_step = ts.TimeStep(
        step_type=np.expand_dims(time_step.step_type, axis=0),
        reward=np.expand_dims(time_step.reward, axis=0),
        discount=np.expand_dims(time_step.discount, axis=0),
        observation=observation)
    
    action_step = policy.action(adjusted_time_step)
    
    # Executa a ação no ambiente para obter o próximo timestep
    next_time_step = environment.step(action_step.action)
    
    # Cria uma trajetória da transição e adiciona ao replay buffer.
    traj = trajectory.from_transition(adjusted_time_step, action_step, next_time_step)
    buffer.add_batch(traj)


In [13]:
print("Especificação de observação do ambiente:", environment.observation_spec())


Especificação de observação do ambiente: BoundedArraySpec(shape=(9,), dtype=dtype('float32'), name='observation', minimum=-3.4028234663852886e+38, maximum=3.4028234663852886e+38)


In [21]:
import numpy as np
from tf_agents.environments import py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.replay_buffers import TFUniformReplayBuffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common


def collect_step(environment, policy, buffer):
    time_step = environment.current_time_step()
    # Não é necessário adicionar manualmente a dimensão do batch se o ambiente já o faz.
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    buffer.add_batch(traj)




from tf_agents.replay_buffers import TFUniformReplayBuffer

# Supondo que 'agent' já foi criado.
# Inicialização do replay buffer
replay_buffer = TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=1,
    max_length=100000)



environment.reset()

# Coletar experiências iniciais
for _ in range(1000):
    collect_step(environment, agent.collect_policy, replay_buffer)



# Configuração do dataset para treinamento
batch_size = 64  # Definir conforme necessário para o treinamento
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3, 
    sample_batch_size=batch_size, 
    num_steps=2).prefetch(3)

# Continuação do treinamento...


iterator = iter(dataset)

# Treinamento do agente
num_iterations = 20000  # Definir o número de iterações de treinamento
for i in range(num_iterations):
    # Opcional: Coletar mais experiências durante o treinamento
    if i % 100 == 0:
        for _ in range(10):  # Número de passos adicionais a coletar a cada 100 iterações
            collect_step(environment, agent.collect_policy, replay_buffer)
    
    # Treinar o agente com experiências do replay buffer
    experience, unused_info = next(iterator)
    train_loss = agent.train(experience).loss
    
    # Opcional: Imprimir o progresso
    if i % 1000 == 0:
        print(f'Iteração {i}, Perda: {train_loss}')

# Adicione o código necessário para salvar o modelo ou avaliar o agente após o treinamento, se necessário.


ValueError: Exception encountered when calling layer 'QNetwork' (type QNetwork).

Input 0 of layer "dense_8" is incompatible with the layer: expected min_ndim=2, found ndim=1. Full shape received: (50,)

Call arguments received by layer 'QNetwork' (type QNetwork):
  • observation=tf.Tensor(shape=(9,), dtype=float32)
  • step_type=array(0)
  • network_state=()
  • training=False